# LinkedIn Job Postings Analysis

Not completed yet! Things written here are subject to change.

In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        # local = single thread
        # local[*] = max threads
        .master('local[*]')
        .appName('Spark Test App')
        # Hive must to be enabled to connect from Metabase
        .enableHiveSupport()
        .getOrCreate()
)

23/10/09 14:29:29 WARN Utils: Your hostname, plasma resolves to a loopback address: 127.0.1.1; using 192.168.1.21 instead (on interface wlan0)
23/10/09 14:29:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/09 14:29:41 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
# View current tables and databases (if any)
print(spark.catalog.listDatabases())
print(spark.catalog.listTables())

[Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='file:/run/media/dhika/Windows/Users/Dhika/Documents/Projects/Proto/Python/Spark/spark-warehouse')]
[]


## Read data from source files

In [4]:
import os
import pandas as pd

files = [
    'data/job_postings',
    'data/company_details/companies',
    'data/company_details/company_industries',
    'data/company_details/company_specialities',
    'data/company_details/employee_counts',
    'data/job_details/benefits',
    'data/job_details/job_industries',
    'data/job_details/job_skills'
]

headers = {}

# It seems that PySpark can't read complex CSV files correctly
# Though Pandas seems able to read them just fine
# As workaround, I use Pandas to export them to JSON first
for file in files:
    if os.path.isfile(f'{file}.csv'):
        # Force all column types as string
        # Auto conversion is inaccurate
        csv = pd.read_csv(f'{file}.csv', dtype = str)
        csv.to_json(f'{file}.json', orient = 'records')
        # Save the column order for later
        headers[f'{file}.json'] = list(csv.columns)

## Get table overview and join related tables

In [5]:
# Get table columns overview
# Kind of useful to see relationship between tables
for key, value in headers.items():
    print(f'* {key} *')
    print(value)
    print()

* data/job_postings.json *
['job_id', 'company_id', 'title', 'description', 'max_salary', 'med_salary', 'min_salary', 'pay_period', 'formatted_work_type', 'location', 'applies', 'original_listed_time', 'remote_allowed', 'views', 'job_posting_url', 'application_url', 'application_type', 'expiry', 'closed_time', 'formatted_experience_level', 'skills_desc', 'listed_time', 'posting_domain', 'sponsored', 'work_type', 'currency', 'compensation_type']

* data/company_details/companies.json *
['company_id', 'name', 'description', 'company_size', 'state', 'country', 'city', 'zip_code', 'address', 'url']

* data/company_details/company_industries.json *
['company_id', 'industry']

* data/company_details/company_specialities.json *
['company_id', 'speciality']

* data/company_details/employee_counts.json *
['company_id', 'employee_count', 'follower_count', 'time_recorded']

* data/job_details/benefits.json *
['job_id', 'inferred', 'type']

* data/job_details/job_industries.json *
['job_id', 'indu

In [6]:
job_postings = 'data/job_postings.json'
df_jp = spark.read.json(job_postings)

# Reading CSV directly using PySpark may cause issues
# Single multiline row can be mistreated as multiple rows
""" # header = get column name from CSV header
# inferSchema = auto detect column data type when possible
df_jp = (
    spark.read.options(
        header = True,
        inferSchema = True
    ).csv('data/job_postings.csv')
) """

# Restore custom column order (the default is ascending)
df_jp = df_jp.select(*headers[job_postings])

# Print column names and types
# df_jp.printSchema()

print(df_jp.count())
print(df_jp.distinct().count())

23/10/08 01:07:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


15886


15886


In [8]:
# Treat current CSV as temporary SQL table
# You can then execute SQL commands on this table
df_jp.createOrReplaceTempView("job_postings")

# We can also use "select distinct" to return only unique rows,
# but all rows are already unique based on previous result
spark.sql('CREATE VIEW job_postings AS SELECT * FROM job_postings').show()
# spark.sql('SELECT * FROM job_postings').show()
# spark.sql('SELECT COUNT(*) FROM job_postings').show()

AnalysisException: [INVALID_TEMP_OBJ_REFERENCE] Cannot create the persistent object `spark_catalog`.`default`.`job_postings` of the type VIEW because it references to the temporary object `job_postings` of the type VIEW. Please make the temporary object `job_postings` persistent, or make the persistent object `spark_catalog`.`default`.`job_postings` temporary.

In [84]:
# # Unimportant columns
# filter_cols = [
#     'description',
#     'original_listed_time',
#     'application_url',
#     'job_posting_url',
#     'expiry',
#     'closed_time',
#     'listed_time',
#     'posting_domain',
#     'sponsored',
#     'work_type'
# ]

# # FIXME The "except" SQL query doesn't work
# # Using pythonic way "not in" as workaround
# filter_cols = [ i for i in df_jp.schema.names if i not in filter_cols ]
# filter_cols = ','.join(filter_cols)

# spark.sql(f'SELECT {filter_cols} FROM job_postings').toPandas()

In [ ]:
spark.sql(
"""CREATE TABLE jobs (
    id Int,
    company String,
    industry String,
    title String,
    experience String,
    salary Int,
    currency String,
    work_type String,
    location String,
    benefit String
)""")

In [80]:
spark.sql('SELECT * FROM job_postings WHERE LOWER(title) LIKE "%data%"').toPandas()

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type
0,3586162459,69642092,Teradata Developer,Duration: 6-12+ Months\nOverview of Role:Indiv...,None,None,None,None,Contract,United States,...,1.70E+12,None,None,None,1.69E+12,None,0,CONTRACT,None,None
1,3690692186,61242,Seasonal Payroll/Data Entry Clerk,Universal Screen Arts' specialty is marketing ...,None,None,None,None,Temporary,"Hudson, OH",...,1.71E+12,None,None,None,1.69E+12,None,0,TEMPORARY,None,None
2,3691795980,7573454,Data Engineer,"Job Description:\n• Design, develop, and launc...",None,None,None,None,Contract,United States,...,1.70E+12,None,None,None,1.69E+12,None,0,CONTRACT,None,None
3,3692302089,37768,Data Scientist/ Product Analyst,Looking for candidates with 4+ years’ experien...,80,None,70,HOURLY,Contract,"San Francisco, CA",...,1.70E+12,None,Mid-Senior level,None,1.69E+12,None,0,CONTRACT,USD,BASE_SALARY
4,3692363778,2474970,Data Analytics Consultant,About the CompanyDiLytics is a leading Informa...,None,None,None,None,Full-time,"Sacramento, CA",...,1.70E+12,None,None,None,1.69E+12,None,0,FULL_TIME,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,3701369746,39203,Data Scientist / Operations Research Analyst,LinQuest is seeking a Data Scientist / Operati...,160000,None,100000,YEARLY,Full-time,"Colorado Springs, CO",...,1.70E+12,None,Entry level,None,1.69E+12,recruiting2.ultipro.com,0,FULL_TIME,USD,BASE_SALARY
377,3701371901,2848937,Data Engineering Product Lead (Hybrid),Job Description\nThe Data Engineering Product ...,304500,None,193440,YEARLY,Full-time,"West Point, PA",...,1.70E+12,None,None,None,1.69E+12,jobs.msd.com,1,FULL_TIME,USD,BASE_SALARY
378,3701372446,2113831,OCM Data Analyst (Remote),GovCIO is looking for an experienced Data Anal...,88000,None,85000,YEARLY,Full-time,"Fairfax, VA",...,1.70E+12,None,Entry level,None,1.69E+12,careers-govcio.icims.com,0,FULL_TIME,USD,BASE_SALARY
379,3701372789,163578,Senior Supply Chain Data Analyst,Job Description:\nOur Sr. Supply Chain Data An...,108000,None,76000,YEARLY,Full-time,"Irvine, CA",...,1.70E+12,None,None,None,1.69E+12,edwards.wd5.myworkdayjobs.com,1,FULL_TIME,USD,BASE_SALARY


In [81]:
# tables = [ i.name for i in spark.catalog.listTables() ]
# print(tables)

# for table in tables:
#     spark.catalog.dropTempView(table)